# Model Training 

## Feature Enginneering

In [33]:
# Importing all required library
import pandas as pd
import numpy as np
import seaborn as sns
from math import radians, cos, sin, asin, acos, sqrt, pi
from geopy import distance
from geopy.geocoders import Nominatim
import osmnx as ox
import networkx as nx
from statistics import  mode
import matplotlib.pyplot as plt

In [34]:
df=pd.read_csv('/config/workspace/Delivery_price_prediction/notebook/data/finalTrain.csv')

In [35]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)'],
      dtype='object')

In [36]:
# Splitting independant and dependant features 
X=df.drop('Time_taken (min)',axis=1)
y=df['Time_taken (min)']

In [37]:
X

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35,11:45,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian
45580,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55,20:10,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian
45581,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50,24:05:00,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian
45582,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35,13:40,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian


In [38]:
# spliting all numerical and categorical features
numerical_fea=X.select_dtypes(exclude='O')
categorical_fea= X.select_dtypes(include="O")

In [39]:
categorical_fea.columns

Index(['ID', 'Delivery_person_ID', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Type_of_order', 'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')

In [40]:
#Categories for ordinal encoding
Weather_conditions_ODE=['Sunny','Cloudy','Windy','Fog', 'Stormy', 'Sandstorms' ]
Road_traffic_density_ODE=['Low', 'Medium', 'High', 'Jam']
Type_of_vehicle_ODE=['bicycle', 'electric_scooter', 'scooter', 'motorcycle']
Festival_ODE=['No','Yes']

#Categories for One Hot encoding
OHE_Cat_City=['Metropolitian', 'Urban', 'Semi-Urban']
OHE_Cat_type_orders=['Snack', 'Meal', 'Drinks', 'Buffet']

In [41]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.preprocessing import OneHotEncoder # One Hot Encoding
## pipelines
from sklearn.pipeline import Pipeline,FunctionTransformer
from sklearn.compose import ColumnTransformer

In [42]:
# Defining numerical and categorical colums
numerical_cols=numerical_fea.columns
categorical_cols=categorical_fea.columns

In [43]:
# Distance calculator

def calculate_spherical_distance(lat1, lon1, lat2, lon2, r=6371):
    
    # Convert degrees to radians
    coordinates = lat1, lon1, lat2, lon2
    # radians(c) is same as c*pi/180
    phi1, lambda1, phi2, lambda2 = [
        radians(c) for c in coordinates
    ]  
    
    # Apply the haversine formula
    a = (np.square(sin((phi2-phi1)/2)) + cos(phi1) * cos(phi2) * 
         np.square(sin((lambda2-lambda1)/2)))
    d = 2*r*asin(np.sqrt(a))
    return d

In [44]:
#Distance Transformation


distancepreprocessing = Pipeline([
    ('distance', FunctionTransformer(lambda x: x.assign(Distance=[round(calculate_spherical_distance(*row), 2) 
    for row in x[['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']].values]))),
    ('drop_cols', FunctionTransformer(lambda x: x.drop(['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude'], axis=1)))
])

# Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
   
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
        
        ] 
                    )
# Categorical Pipeline
cat_pipeline_ODE =Pipeline(
    steps=[
    
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[Weather_conditions_ODE,Road_traffic_density_ODE,Type_of_vehicle_ODE,Festival_ODE])),
    ('scaler',StandardScaler())
    
    ]
                    )
cat_pipeline_OHE=Pipeline(
    steps=[
        ('onehotencoder',OneHotEncoder(categories=[OHE_Cat_City,OHE_Cat_type_orders]))
    ]
)


preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline_ODE',cat_pipeline_ODE,['Weather_conditions', 'Road_traffic_density','Type_of_vehicle', 'Festival']),
('cat_pipeline_OHE',cat_pipeline_OHE,['City','Type_of_order'])
])

In [45]:
# # Fit and transform the pipeline on the input dataframe
X = distancepreprocessing.fit_transform(X)


In [74]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)



In [75]:
X_train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Distance
5755,0x2a7b,VADRES010DEL01,36.0,4.1,02-03-2022,21:40,21:50,Sandstorms,Jam,0,Buffet,motorcycle,0.0,No,Metropolitian,10.60
9163,0x787a,VADRES05DEL03,29.0,3.6,19-03-2022,21:10,21:15,Sunny,Jam,0,Buffet,motorcycle,3.0,Yes,Urban,9.09
16410,0xd96c,KNPRES09DEL02,31.0,4.8,15-02-2022,10:45,10:55,Windy,Low,2,Meal,scooter,1.0,No,Metropolitian,2.98
36945,0xdd9e,GOARES14DEL02,33.0,4.2,14-02-2022,19:15,19:30,Windy,Jam,2,Meal,scooter,1.0,No,Metropolitian,20.09
14679,0x92b2,MUMRES01DEL02,34.0,4.1,09-03-2022,08:45,0.375,Windy,Low,0,Meal,motorcycle,1.0,No,Urban,3.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,0xd7aa,KNPRES11DEL02,37.0,4.9,17-02-2022,08:25,08:35,Sunny,Low,1,Snack,motorcycle,0.0,No,Urban,2.98
33268,0x2db2,HYDRES13DEL02,28.0,4.9,01-04-2022,17:40,17:45,Fog,Medium,2,Buffet,motorcycle,0.0,No,Metropolitian,7.68
44845,0xbf5d,KOLRES02DEL02,20.0,NaN,14-02-2022,22:20,22:25,Fog,Low,2,Drinks,scooter,1.0,No,Metropolitian,12.11
4517,0x213e,BANGRES05DEL02,34.0,4.6,03-04-2022,08:55,09:10,Cloudy,Low,1,Buffet,motorcycle,NaN,No,Metropolitian,3.11


In [82]:
preprocessor.get_feature_names_out()

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [76]:
#Feature Engineering
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())


ValueError: A given column is not a column of the dataframe

In [ ]:
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [ ]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [ ]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 6.70099488772049
MAE: 5.286610778163107
R2 score 48.085581687588665


Lasso
Model Training Performance
RMSE: 7.12126734980679
MAE: 5.667943883855022
R2 score 41.36944569466444


Ridge
Model Training Performance
RMSE: 6.7010193182947955
MAE: 5.286636246366058
R2 score 48.085203146375775


Elasticnet
Model Training Performance
RMSE: 7.228963921534114
MAE: 5.778484426460662
R2 score 39.58266933394644


